In [1]:
from setup_general import *

# xg boost


In [2]:
train = train_est_smote100_full.copy()
val = val_est_prepared.copy()

X_train = train.drop('type', axis=1)
y_train = train.type

X_val = val.drop('type', axis=1)
y_val = val.type

label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y_train)

y_train = label_encoder.transform(y_train)
y_val = label_encoder.transform(y_val)


In [12]:
data = train_prep.copy()
rebalancing = False

features = data.drop('type', axis=1)
labels = data.type
# at least xgboost cannot deal with string labels
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(labels)
labels = label_encoder.transform(labels)
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.3, random_state=0)

# possible rebalancing

In [ ]:
if rebalancing == True:
    from imblearn.over_sampling import SMOTE, ADASYN
    from collections import Counter

    def strat(data):
        b = Counter(data).values()
        a = Counter(data).keys()
        a = list(a)
        b = list(b)

        for i in range(len(a)):
            if b[i] < 100 :
                b[i] = 100
        return dict(zip(a, b))

    X_train, y_train = SMOTE(sampling_strategy=strat(y_train), random_state=1).fit_resample(X_train, y_train)

# Classifier

In [7]:
bst = XGBClassifier(random_state=0)
# fit model
bst.fit(X_train, y_train)
# make predictions
preds = bst.predict(X_val)
val_acc = accuracy_score(y_val, preds)
val_acc

0.9788095238095238

In [17]:
preds = bst.predict(X_val)
val_acc = accuracy_score(y_val, preds)
val_acc
# 0.9007142857142857 normal 
# 0.900952380952381 est smote

0.900952380952381

In [8]:
bst.save_model('models/xg/xg_est_data_smote100_full.json')

# submission from model

In [86]:
test_set = test_prep.drop('type', axis=1)
results = bst.predict(test_set)
submission = pd.DataFrame({'id': test_set.index ,'type': bst.predict(test_set)})
submission = submission.replace(type_lookup.id.to_list(), type_lookup.estonian.to_list())
submission.to_csv('data/submission_xgboost.csv', index=False)